### Hw-2: Task 2
### Taslima Akter
### ID: takter

In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from operator import itemgetter
from sklearn.datasets import fetch_mldata

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical

%matplotlib notebook

In [2]:
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.33

### Read file names for training data

In [3]:
### File names for Training Data

import glob
import librosa
fname_trn=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trn*.wav'):
    (fname_trn.append(filename))
fname_trn.sort()
# fname_trn

fname_trs=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trs*.wav'):
    (fname_trs.append(filename))
fname_trs.sort()
# fname_trs

fname_trx=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trx*.wav'):
    (fname_trx.append(filename))
fname_trx.sort()
fname_trx[100]


'/opt/e533/timit-homework/tr/trx0100.wav'

### Read file names for validation data

In [4]:
### File names for validation Data

fname_val_n=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vn*.wav'):
    (fname_val_n.append(filename))
fname_val_n.sort()
# print(fname_val_n)

fname_val_s=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vs*.wav'):
    (fname_val_s.append(filename))
fname_val_s.sort()
# fname_trs

fname_val_x=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vx*.wav'):
    (fname_val_x.append(filename))
fname_val_x.sort()


### Read file names for test data

In [5]:
### File names for Test Data

fname_test=[]
for filename in glob.glob('/opt/e533/timit-homework/te/tex*.wav'):
    (fname_test.append(filename))
fname_test.sort()


### Write data into txt file

In [8]:
### Function For writing file
import librosa
def write_file(file_name, fname_list):
    ### Writing training data S

    with open(file_name, 'wb') as fs:
        for i in range(len(fname_list)):
            sn, sr=librosa.load(fname_list[i], sr=None)
            Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
            mag_Sn=np.abs(Sn)
    #         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
            np.savetxt(fs, mag_Sn, fmt='%.5f')
            fs.write(b'\n')
    fs.close()            

    

### Writing X into file 

In [9]:
### Function For writing just X into file
import librosa
def write_file_X(file_name, fname_list):
    ### Writing training data S

    with open(file_name, 'wb') as fs:
        for i in range(len(fname_list)):
            sn, sr=librosa.load(fname_list[i], sr=None)
            Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
#             mag_Sn=np.abs(Sn)
    #         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
            np.savetxt(fs, Sn, fmt='%.5f')
            fs.write(b'\n')
    fs.close()            

    

In [41]:
### Writing training data S
Train_complx_X=[]

for i in range(len(fname_trx)):
    print(i)
    
    sn, sr=librosa.load(fname_trx[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    Train_complx_X.append(np.array(Sn))


#             mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [45]:
### Writing training data S
val_complx_X=[]

for i in range(len(fname_val_x)):
    print(i)
    
    sn, sr=librosa.load(fname_val_x[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    val_complx_X.append(np.array(Sn))


#             mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [88]:
### Writing training data S
val_complx_S=[]

for i in range(len(fname_val_s)):
    print(i)
    
    sn, sr=librosa.load(fname_val_s[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    val_complx_S.append(np.array(Sn))


#             mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [90]:
# len(Train_complx_X[0].shape)
# Train_complx_X[0]

In [47]:
len(val_complx_X)

1200

In [6]:
### Write validation files

write_file("validation_s.txt", fname_val_s)
write_file("validation_n.txt", fname_val_n)
write_file("validation_x.txt", fname_val_x)


In [8]:
write_file_X("train_x_tr.txt", fname_trx)


In [9]:
write_file_X("validation_x_tr.txt", fname_trx)


In [8]:
write_file("test_data.txt", fname_test)

In [9]:
### Writing training data N
import librosa

count=0
total_train_s=[]
with open('train_n.txt', 'wb') as fn:
    for i in range(len(fname_trn)):
        sn, sr=librosa.load(fname_trn[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fn, mag_Sn, fmt='%.5f')
        fn.write(b'\n')
fn.close()            

In [10]:
### Writing training data S

with open('train_s.txt', 'wb') as fs:
    for i in range(len(fname_trs)):
        sn, sr=librosa.load(fname_trs[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fs, mag_Sn, fmt='%.5f')
        fs.write(b'\n')
fs.close()            


In [11]:
### Writing training file X

with open('train_x.txt', 'wb') as fs:
    for i in range(len(fname_trx)):
        sn, sr=librosa.load(fname_trx[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fs, mag_Sn, fmt='%.5f')
        fs.write(b'\n')
fs.close()            


### Read data from files

In [10]:
### Function for Reading file

def read_file(file_name):
    with open(file_name) as f:
        lines=f.readlines()
        print(len(lines))
        sentence_full=[]
        count = 0
        sentence=[]
        for line in lines:

            if count < 513:
                if count ==0:
                    sentence=np.array(np.fromstring(line, dtype=complex_, sep=' '), ndmin=2)
                    count+=1
                else:
                    myarray = np.array(np.fromstring(line, dtype=complex_, sep=' '), ndmin=2)
                    sentence=np.concatenate((sentence, myarray), axis=0)
                    count+=1
            else:
                sentence_full.append(sentence) 
                count=0
                sentence=[]
        return sentence_full


In [50]:
data_train_n = read_file("train_n.txt")
data_train_s = read_file("train_s.txt")
data_train_x = read_file("train_x.txt")

616800
616800
616800


In [53]:
data_train_s[0].shape

(513, 65)

### Read validation data

In [51]:
data_val_n = read_file("validation_n.txt")
data_val_s = read_file("validation_s.txt")
data_val_x = read_file("validation_x.txt")

616800
616800
616800


### Read X from training and validation data

In [23]:
data_train_xtr = read_file("train_x_tr.txt")
data_val_xtr = read_file("validation_x_tr.txt")

616800
616800


In [29]:
len(data_val_xtr)
data_val_xtr[0].shape

(513, 1)

In [54]:
data_test = read_file("test_data.txt")

205600


### Calculating M for training and validation data

In [55]:
### Calculating M:
data_train_M=[]
data_val_M=[]
for i in range(len(data_train_s)):
    data_train_M.append(1*(data_train_s[i]>data_train_n[i]))
    data_val_M.append(1*(data_val_s[i]>data_val_n[i]))
    
    

In [61]:
sum(data_train_M[2])

array([  0,   0,  99, 110, 147, 217, 312, 336, 230, 261, 235, 228, 190,
       206, 284, 315, 238, 195, 219, 169, 342, 362, 319, 299, 345, 317,
       273, 264, 291, 229, 203, 181, 199, 214, 203, 171, 117, 133, 217,
       247, 289, 228, 158, 164, 256, 306, 271, 332, 350, 277, 188, 139,
       258, 251, 215, 108,  23,  17, 153, 147, 142, 141, 128, 131, 129])

In [64]:
data_train_M[2].shape

(513, 65)

### Creating Batch  

In [36]:
feature_vec=None
def next_batch(data, data_M):
    feature_vec=None
    count=0
    for i in range(1200):
        if count==10:
            yield feature_vec
        temp=data[i].T
        temp_M=data_M[i].T
        tuple_vec=(temp, temp_M)
        feature_vec = np.array(tuple_vec) if feature_vec is None else np.concatenate( (feature_vec,tuple_vec),axis=1)
#         M_vec = np.array(temp_M) if M_vec is None else np.concatenate( (M_vec,temp_M),axis=1)
        count+=1
        print(feature_vec.shape)
#         yield feature_vec
        
        

In [97]:
def next_batchXSCmplx(X_, S_, X_cmplx, S_cmplx):
    
    batch_x = None
    batch_s = None
    batch_x_cmplx = None
    batch_s_cmplx = None
    
    for e,(x, s, x_cmplx, s_cmplx) in enumerate(zip(X_, S_, X_cmplx, S_cmplx)): 
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_s = np.array(s.T) if batch_s is None else np.concatenate( (batch_s,s.T), axis=0)
        batch_x_cmplx = np.array(x_cmplx.T) if batch_x_cmplx is None else np.concatenate( (batch_x_cmplx,x_cmplx.T), axis=0)
        batch_s_cmplx = np.array(s_cmplx.T) if batch_s_cmplx is None else np.concatenate( (batch_s_cmplx,s_cmplx.T), axis=0)
 
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_s, batch_s = batch_s, None
            temp_x_cmplx, batch_x_cmplx = batch_x_cmplx, None
            temp_s_cmplx, batch_s_cmplx = batch_s_cmplx, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_s = temp_s.reshape((-1,Max_RNN,513))
            temp_x_cmplx = temp_x_cmplx.reshape((-1,Max_RNN,513))
#             temp_s_cmplx = temp_s_cmplx.reshape((-1,Max_RNN,513))

            yield temp_x, temp_s, temp_x_cmplx, temp_s_cmplx

In [ ]:
def next_batchX(X_):
    
    batch_x = None
    
    for e,x in enumerate(X_): 
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))

            yield temp_x

In [56]:
def next_batch(X_,Y_):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
#         print(e)
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)
        
#         print('batch_x',batch_x.shape)
#         print('batch_y',batch_y.shape)
        
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_y, batch_y = batch_y, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))

#             print('temp_x',temp_x.shape)
#             print('temp_y',temp_y.shape)
        
            yield temp_x,temp_y


### RNN implementation

In [ ]:
Max_RNN=5
model = Sequential()

model.add(Bidirectional(LSTM(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(Max_RNN, return_sequences=True)))
# model.add(GRU(output_dim = 513, input_length = 5, input_dim = 513, return_sequences=True))

# model.add(Activation('relu'))
# model.add(TimeDistributed(Dense(513, activation='sigmoid')))
model.add(Dense(513, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

for e in range(10):
    for (b_x,b_y), (v_x,v_y) in zip(next_batch(data_train_x, data_train_M), next_batch(data_val_x, data_val_M)):
        model.fit(b_x, b_y, validation_data=(v_x,v_y), shuffle=True, batch_size=100)

#     model.fit( , epochs=20, steps_per_epoch=700, validation_data=next_batch_mb(DATA_val_x, DATA_val_x,10), validation_steps=700)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_9 (Bidirection (None, 5, 10)             20760     
_________________________________________________________________
dropout_5 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 5, 10)             640       
_________________________________________________________________
dense_5 (Dense)              (None, 5, 513)            5643      
Total params: 27,043
Trainable params: 27,043
Non-trainable params: 0
_________________________________________________________________
None
Train on 130 samples, validate on 130 samples
Epoch 1/1
130/130 [==============================] - 3s 22ms/step - loss: 0.6931 - acc: 0.5045 - val_loss: 0.6929 - val_acc: 0.5286
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [======================

Train on 210 samples, validate on 210 samples
Epoch 1/1
210/210 [==============================] - 0s 740us/step - loss: 0.6770 - acc: 0.5557 - val_loss: 0.6715 - val_acc: 0.5570
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 668us/step - loss: 0.6558 - acc: 0.6350 - val_loss: 0.6667 - val_acc: 0.5627
Train on 198 samples, validate on 198 samples
Epoch 1/1
198/198 [==============================] - 0s 601us/step - loss: 0.6563 - acc: 0.6127 - val_loss: 0.6635 - val_acc: 0.5837
Train on 206 samples, validate on 206 samples
Epoch 1/1
206/206 [==============================] - 0s 744us/step - loss: 0.6847 - acc: 0.5008 - val_loss: 0.6723 - val_acc: 0.5474
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 646us/step - loss: 0.6611 - acc: 0.5799 - val_loss: 0.6523 - val_acc: 0.6077
Train on 214 samples, validate on 214 samples
Epoch 1/1
214/214 [==============================] - 0s 747

Train on 228 samples, validate on 228 samples
Epoch 1/1
228/228 [==============================] - 0s 687us/step - loss: 0.6811 - acc: 0.5430 - val_loss: 0.6607 - val_acc: 0.6203
Train on 202 samples, validate on 202 samples
Epoch 1/1
202/202 [==============================] - 0s 757us/step - loss: 0.6468 - acc: 0.6444 - val_loss: 0.6447 - val_acc: 0.6357
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 598us/step - loss: 0.6570 - acc: 0.5971 - val_loss: 0.6479 - val_acc: 0.6475
Train on 242 samples, validate on 242 samples
Epoch 1/1
242/242 [==============================] - 0s 734us/step - loss: 0.7030 - acc: 0.5378 - val_loss: 0.6988 - val_acc: 0.5683
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [==============================] - 0s 695us/step - loss: 0.6511 - acc: 0.6339 - val_loss: 0.6502 - val_acc: 0.6327
Train on 356 samples, validate on 356 samples
Epoch 1/1
356/356 [==============================] - 0s 617

Train on 154 samples, validate on 154 samples
Epoch 1/1
154/154 [==============================] - 0s 687us/step - loss: 0.6252 - acc: 0.6312 - val_loss: 0.6186 - val_acc: 0.6789
Train on 262 samples, validate on 262 samples
Epoch 1/1
262/262 [==============================] - 0s 662us/step - loss: 0.6648 - acc: 0.5967 - val_loss: 0.6442 - val_acc: 0.6556
Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 678us/step - loss: 0.6021 - acc: 0.7044 - val_loss: 0.5761 - val_acc: 0.7391
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 670us/step - loss: 0.6434 - acc: 0.6392 - val_loss: 0.6698 - val_acc: 0.6402
Train on 188 samples, validate on 188 samples
Epoch 1/1
188/188 [==============================] - 0s 617us/step - loss: 0.6164 - acc: 0.6907 - val_loss: 0.6093 - val_acc: 0.6994
Train on 216 samples, validate on 216 samples
Epoch 1/1
100/216 [============>.................] - ETA: 0

### Calculating Accuracy

In [86]:
scores = []
for v_x,v_y in next_batch(data_val_x, data_val_M):
    scores.append( model.evaluate(v_x, v_y,verbose=0)[1] )

print("Accuracy: %.2f%%" % np.mean(scores*100))

Accuracy: 0.75%


### Calculating SNR

In [102]:
sum_s = 0.0
sum_s_diff = 0.0

for v_s,v_x,v_x_cmplx,v_s_cmplx in next_batchXSCmplx(data_val_s,data_val_x,val_complx_X,val_complx_S):
    
    print(v_s.shape)
    print(v_x.shape)
    print(v_x_cmplx.shape)
    print('v_s_cmplx',v_s_cmplx.shape)
    
    mask = model.predict(v_x)
    S_hat = (mask) * v_x_cmplx
    S_hat = S_hat.reshape(-1,513).T
    S = v_s_cmplx.T
#     S=S.reshape(-1,513)
    
    print('S.shape',S.shape)
    print('S_hat.shape',S_hat.shape)

    S_org = librosa.istft(S, hop_length=512)
    S_pred = librosa.istft(S_hat, hop_length=512)

    sum_s += np.sum(S_org*S_org)
    sum_s_diff += np.sum((S_org-S_pred)*(S_org-S_pred))
    
acc = sum_s/ sum_s_diff
print(acc)

10*np.log10(acc)

(130, 5, 513)
(130, 5, 513)
(130, 5, 513)
v_s_cmplx (650, 513)
S.shape (513, 650)
S_hat.shape (513, 650)
(260, 5, 513)
(260, 5, 513)
(260, 5, 513)
v_s_cmplx (1300, 513)
S.shape (513, 1300)
S_hat.shape (513, 1300)
(150, 5, 513)
(150, 5, 513)
(150, 5, 513)
v_s_cmplx (750, 513)
S.shape (513, 750)
S_hat.shape (513, 750)
(138, 5, 513)
(138, 5, 513)
(138, 5, 513)
v_s_cmplx (690, 513)
S.shape (513, 690)
S_hat.shape (513, 690)
(184, 5, 513)
(184, 5, 513)
(184, 5, 513)
v_s_cmplx (920, 513)
S.shape (513, 920)
S_hat.shape (513, 920)
(206, 5, 513)
(206, 5, 513)
(206, 5, 513)
v_s_cmplx (1030, 513)
S.shape (513, 1030)
S_hat.shape (513, 1030)
(174, 5, 513)
(174, 5, 513)
(174, 5, 513)
v_s_cmplx (870, 513)
S.shape (513, 870)
S_hat.shape (513, 870)
(278, 5, 513)
(278, 5, 513)
(278, 5, 513)
v_s_cmplx (1390, 513)
S.shape (513, 1390)
S_hat.shape (513, 1390)
(156, 5, 513)
(156, 5, 513)
(156, 5, 513)
v_s_cmplx (780, 513)
S.shape (513, 780)
S_hat.shape (513, 780)
(160, 5, 513)
(160, 5, 513)
(160, 5, 513)
v_s_

(140, 5, 513)
(140, 5, 513)
(140, 5, 513)
v_s_cmplx (700, 513)
S.shape (513, 700)
S_hat.shape (513, 700)
(180, 5, 513)
(180, 5, 513)
(180, 5, 513)
v_s_cmplx (900, 513)
S.shape (513, 900)
S_hat.shape (513, 900)
(142, 5, 513)
(142, 5, 513)
(142, 5, 513)
v_s_cmplx (710, 513)
S.shape (513, 710)
S_hat.shape (513, 710)
(108, 5, 513)
(108, 5, 513)
(108, 5, 513)
v_s_cmplx (540, 513)
S.shape (513, 540)
S_hat.shape (513, 540)
(184, 5, 513)
(184, 5, 513)
(184, 5, 513)
v_s_cmplx (920, 513)
S.shape (513, 920)
S_hat.shape (513, 920)
(202, 5, 513)
(202, 5, 513)
(202, 5, 513)
v_s_cmplx (1010, 513)
S.shape (513, 1010)
S_hat.shape (513, 1010)
(152, 5, 513)
(152, 5, 513)
(152, 5, 513)
v_s_cmplx (760, 513)
S.shape (513, 760)
S_hat.shape (513, 760)
(174, 5, 513)
(174, 5, 513)
(174, 5, 513)
v_s_cmplx (870, 513)
S.shape (513, 870)
S_hat.shape (513, 870)
(140, 5, 513)
(140, 5, 513)
(140, 5, 513)
v_s_cmplx (700, 513)
S.shape (513, 700)
S_hat.shape (513, 700)
(228, 5, 513)
(228, 5, 513)
(228, 5, 513)
v_s_cmplx 

5.526181781952143

### Write into audio file

In [109]:
def write_audio(file_name):
    for i in range(len(file_name)):
        audio_fname=file_name[i].replace("/opt/e533/timit-homework/te/", "").replace(".wav","")
        print(audio_fname)
        sn, sr=librosa.load(file_name[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
        print(mag_Sn.shape)
#         mag_Sn=mag_Sn.reshape(-1, 5, 513)
        Stest_hat=model.predict(mag_Sn.reshape(-1, 5, 513))
        Stest_hat=Stest_hat.reshape(-1,513)
        S_hat=(Sn/mag_Sn)*Stest_hat.T

        S_time=librosa.istft(S_hat, hop_length=512)
        audio_fname=audio_fname + "_recons.wav"
        print(audio_fname)
        librosa.output.write_wav(audio_fname, S_time, sr)
        
        
    

In [110]:
write_audio(fname_test[:10])

tex0000
(513, 110)
tex0000_recons.wav
tex0001
(513, 110)
tex0001_recons.wav
tex0002
(513, 110)
tex0002_recons.wav
tex0003
(513, 110)
tex0003_recons.wav
tex0004
(513, 110)
tex0004_recons.wav
tex0005
(513, 110)
tex0005_recons.wav
tex0006
(513, 110)
tex0006_recons.wav
tex0007
(513, 110)
tex0007_recons.wav
tex0008
(513, 110)
tex0008_recons.wav
tex0009
(513, 110)
tex0009_recons.wav


In [18]:
import librosa

sn, sr=librosa.load(fname_trn[0], sr=None)
Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
trn_arr=np.abs(Sn)
print(trn_arr.shape)
sx, sr=librosa.load(fname_trx[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
trx_arr=np.abs(Sx)

ss, sr=librosa.load(fname_trs[0], sr=None)
Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
trs_arr=np.abs(Ss)
for i in range(1,1200):

    sn, sr=librosa.load(fname_trn[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    mag_Sn=np.abs(Sn)
    trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
#     print(Sn.shape)

    sx, sr=librosa.load(fname_trx[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
    mag_Sx=np.abs(Sx)
    trx_arr=np.concatenate((trx_arr, mag_Sx), axis=1)
    
    ss, sr=librosa.load(fname_trs[i], sr=None)
    Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
    mag_Ss=np.abs(Ss)
    trs_arr=np.concatenate((trs_arr, mag_Ss), axis=1)
# print(Sn.shape)

# sn, sr=librosa.load(fname_trn[129], sr=None)
# Sn=librosa.stft(sn, n_fft=1024, hop_length=512)

# print(Sn.shape)

(513, 65)


In [20]:
DATA_train_M = 1*(trs_arr>trn_arr)
DATA_train_M.shape

(513, 118550)

In [40]:
trn_arr = trn_arr.reshape( (513,10,-1))
trx_arr = trx_arr.reshape( (513,10,-1))
trs_arr = trs_arr.reshape( (513,10,-1))
DATA_train_M = DATA_train_M.reshape( (513,10,-1))

trn_arr.shape, trx_arr.shape, trs_arr.shape, DATA_train_M.shape


((513, 10, 11855), (513, 10, 11855), (513, 10, 11855), (513, 10, 11855))

In [14]:
import librosa

sn, sr=librosa.load(fname_val_n[0], sr=None)
Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
val_n_arr=np.abs(Sn)

# print(trn_arr.shape)
sx, sr=librosa.load(fname_val_x[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
val_x_arr=np.abs(Sx)

ss, sr=librosa.load(fname_val_s[0], sr=None)
Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
val_s_arr=np.abs(Ss)
for i in range(1,len(fname_val_n)):

    sn, sr=librosa.load(fname_val_n[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    mag_Sn=np.abs(Sn)
    val_n_arr=np.concatenate((val_n_arr, mag_Sn), axis=1)
#     print(Sn.shape)

    sx, sr=librosa.load(fname_val_x[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
    mag_Sx=np.abs(Sx)
    val_x_arr=np.concatenate((val_x_arr, mag_Sx), axis=1)
    
    ss, sr=librosa.load(fname_val_s[i], sr=None)
    Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
    mag_Ss=np.abs(Ss)
    val_s_arr=np.concatenate((val_s_arr, mag_Ss), axis=1)
# print(Sn.shape)

# sn, sr=librosa.load(fname_trn[129], sr=None)
# Sn=librosa.stft(sn, n_fft=1024, hop_length=512)

# print(Sn.shape)

In [15]:
DATA_val_M = 1*(val_s_arr>val_n_arr)
DATA_val_M.shape

(513, 118550)

In [41]:
val_n_arr = val_n_arr.reshape( (513,10,-1))
val_x_arr = val_x_arr.reshape( (513,10,-1))
val_s_arr = val_s_arr.reshape( (513,10,-1))
DATA_val_M = DATA_val_M.reshape( (513,10,-1))

val_n_arr.shape, val_x_arr.shape, val_s_arr.shape, DATA_val_M.shape


((513, 10, 11855), (513, 10, 11855), (513, 10, 11855), (513, 10, 11855))

In [8]:
sn, sr=librosa.load(fname_test[0], sr=None)
Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
test_arr=np.abs(Sn)

# print(trn_arr.shape)
# sx, sr=librosa.load(fname_val_x[0], sr=None)
# Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
# val_x_arr=np.abs(Sx)

# ss, sr=librosa.load(fname_val_s[0], sr=None)
# Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
# val_s_arr=np.abs(Ss)
for i in range(1,len(fname_test)):

    sn, sr=librosa.load(fname_test[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    mag_Sn=np.abs(Sn)
    test_arr=np.concatenate((test_arr, mag_Sn), axis=1)


In [42]:
test_arr.shape
test_arr = test_arr.reshape( (513,10,-1))
test_arr.shape

(513, 10, 4415)

In [43]:
np.save('DATA_train_s.npy', trs_arr)
np.save('DATA_train_n.npy', trn_arr)
np.save('DATA_train_x.npy', trx_arr)
np.save('DATA_train_M.npy', DATA_train_M)

np.save('DATA_val_s.npy', val_s_arr)
np.save('DATA_val_n.npy', val_n_arr)
np.save('DATA_val_x.npy', val_x_arr)
np.save('DATA_val_M.npy', DATA_val_M)

np.save('DATA_test_x.npy', test_arr)

In [44]:
DATA_train_s_f = np.load('DATA_train_s.npy')
DATA_train_n_f = np.load('DATA_train_n.npy')
DATA_train_x_f = np.load('DATA_train_x.npy')
DATA_train_M_f = np.load('DATA_train_M.npy')

DATA_val_s_f = np.load('DATA_val_s.npy')
DATA_val_n_f = np.load('DATA_val_n.npy')
DATA_val_x_f = np.load('DATA_val_x.npy')
DATA_val_M_f = np.load('DATA_val_M.npy')

DATA_test_x_f = np.load('DATA_test_x.npy')

In [111]:
# DATA_val_M_f[:10, :3, :10]

In [45]:
tr_categorical_labels = to_categorical(DATA_train_M_f.T, num_classes=2)
print(tr_categorical_labels.shape)

v_categorical_labels = to_categorical(DATA_val_M_f.T, num_classes=2)
print(v_categorical_labels.shape)

(11855, 10, 513, 2)
(11855, 10, 513, 2)


In [54]:
Max_RNN = 10
# create the model

model = Sequential()

model.add(LSTM(Max_RNN, return_sequences=True, input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))

model.add(Dense(513, activation='sigmoid'))
          
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# model.fit(DATA_train_x_f.T, tr_categorical_labels, validation_data=(DATA_val_x_f.T,v_categorical_labels), shuffle=True, nb_epoch=100, batch_size=10)
# # Final evaluation of the model
# scores = model.evaluate(DATA_val_x_f.T, v_categorical_labels, verbose=0)



model.fit(DATA_train_x_f.T, DATA_train_M_f.T, validation_data=(DATA_val_x_f.T,DATA_val_M_f.T), shuffle=True, nb_epoch=25, batch_size=10)
# Final evaluation of the model
scores = model.evaluate(DATA_val_x_f.T, DATA_val_M_f.T, verbose=0)



print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10, 10)            20960     
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 10)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 10, 513)           5643      
Total params: 26,603
Trainable params: 26,603
Non-trainable params: 0
_________________________________________________________________
None


/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 11855 samples, validate on 11855 samples
Epoch 1/25
11855/11855 [==============================] - 20s 2ms/step - loss: 0.2133 - acc: 0.0017 - val_loss: 0.1920 - val_acc: 0.0018
Epoch 2/25
11855/11855 [==============================] - 20s 2ms/step - loss: 0.1944 - acc: 0.0026 - val_loss: 0.1836 - val_acc: 0.0022
Epoch 3/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1891 - acc: 0.0033 - val_loss: 0.1782 - val_acc: 0.0022
Epoch 4/25
11855/11855 [==============================] - 20s 2ms/step - loss: 0.1859 - acc: 0.0049 - val_loss: 0.1751 - val_acc: 0.0035
Epoch 5/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1838 - acc: 0.0076 - val_loss: 0.1735 - val_acc: 0.0071
Epoch 6/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1824 - acc: 0.0124 - val_loss: 0.1718 - val_acc: 0.0091
Epoch 7/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1812 - acc: 0.0162 - val_loss: 0.1712 - val_acc

In [55]:
import librosa

sx, sr=librosa.load(fname_trx[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
X_tr=Sx

for i in range(1,len(fname_trx)):

    sx, sr=librosa.load(fname_trx[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
#     mag_Sx=np.abs(Sx)
    X_tr=np.concatenate((X_tr, Sx), axis=1)
    


In [ ]:
import librosa

sx, sr=librosa.load(fname_val_x[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
X_val=Sx

for i in range(1,len(fname_val_x)):

    sx, sr=librosa.load(fname_val_x[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
#     mag_Sx=np.abs(Sx)
    X_val=np.concatenate((X_val, Sx), axis=1)
    


In [48]:
labels = model.predict(DATA_val_x_f.T)
labels.shape

(11855, 10, 513)

In [49]:
S_hat = (labels.T) * DATA_val_x_f

In [50]:
S_org = (DATA_val_s_f).flatten()
S_pred = S_hat.flatten()

In [51]:
acc = np.sum(S_org*S_org)/ np.sum((S_org-S_pred)*(S_org-S_pred))
print(acc)

10*np.log10(acc)

4.074747


6.101006865501404